In [0]:
import gensim
from google.colab import drive
drive.mount('/content/mydrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/mydrive


In [0]:
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
from numpy import dot
from numpy.linalg import norm
import gensim.models.ldamulticore
import os
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Add
from keras.layers import Average
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import AveragePooling1D
from keras.layers import Lambda
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import regularizers
from keras.backend import mean
from keras.backend import sum

Using TensorFlow backend.


Part 2

In [0]:
path="mydrive/My Drive/labeled_articles/"
#load the files from the drive 
allFiles=[]
#every element of list allFiles contains the same text 3 times from every evaluator
for f in os.listdir(path):
  new=True
  for i in range(len(allFiles)):
    if(allFiles[i][0][:-5]==f[:-5]):
      new=False
      allFiles[i].append(f)
  if(new):
    allFiles.append([f])
X=[]
Y=[]
sentences=[]
for i in range(len(allFiles)):
  sentences=[]
  for j in range(len(allFiles[i])):
    sentences.append([])
    filepath = path+allFiles[i][j]
    with open(filepath) as fp:  
      line = fp.readline()
      while line:
        sentences[j].append(line)
        line = fp.readline()
  #sentences contain 3 lists,every element of those lists contain a sentence from different evaluators
  for k in range(len(sentences[0])):
    #ignore comments
    if(sentences[0][k][0]!='#'):
      #for each sentence we extract the class from the 3 different evaluators
      term1=sentences[0][k][:4]
      term2=sentences[1][k][:4]
      term3=sentences[2][k][:4]
      #if 2 evaluators agree for the class of a sentence we accept it,otherwise we ignore the sentence
      if(term1==term2 or term1==term3):
        X.append(sentences[0][k][4:])
        Y.append(term1)
      elif(term2==term3):
        X.append(sentences[0][k][4:])
        Y.append(term2)
trainData=[]

def readFiles(path,docs):
  count=0
  for f in os.listdir(path):
    filepath=path+f
    with open(filepath) as fp:
      count=count+1
      docs.append(fp.read())
      if(count==100):
        break
  return docs
print(len(X))
X=readFiles("mydrive/My Drive/unlabeled_articles/arxiv_unlabeled/",X)
X=readFiles("mydrive/My Drive/unlabeled_articles/jdm_unlabeled/",X)
X=readFiles("mydrive/My Drive/unlabeled_articles/plos_unlabeled/",X)
print(len(X))
for i in range(len(X)):
  Line=remove_stopwords(X[i])
  trainData.append(gensim.utils.simple_preprocess(Line))

In [0]:
firstModel = gensim.models.Word2Vec(
        trainData,
        size=100,
        window=20,
        min_count=2,
        workers=10)
firstModel.train(trainData, total_examples=len(trainData), epochs=10)

print(len(list(firstModel.wv.vocab)))
secondModel = gensim.models.Word2Vec(
        trainData,
        size=200,
        window=1,
        min_count=2,
        workers=10)
secondModel.train(trainData, total_examples=len(trainData), epochs=10)

print(len(list(secondModel.wv.vocab)))

firstModel.wv.save_word2vec_format("mydrive/My Drive/model_word2vec.kv", binary=False)

In [0]:
listOfWords=firstModel.wv.index2word
for i in range(10):
  print(listOfWords[i])
  print(firstModel.wv.most_similar(positive=listOfWords[i],topn=5))
  print("\n")

listOfWords2=secondModel.wv.index2word
for i in range(10):
  print(listOfWords2[i])
  print(secondModel.wv.most_similar(positive=listOfWords2[i],topn=5))
  print("\n")

Part 3

In [0]:
path="mydrive/My Drive/labeled_articles/"
#load the files from the drive 
allFiles=[]
#every element of list allFiles contains the same text 3 times from every evaluator
for f in os.listdir(path):
  new=True
  for i in range(len(allFiles)):
    if(allFiles[i][0][:-5]==f[:-5]):
      new=False
      allFiles[i].append(f)
  if(new):
    allFiles.append([f])
X=[]
Y=[]
sentences=[]
for i in range(len(allFiles)):
  sentences=[]
  for j in range(len(allFiles[i])):
    sentences.append([])
    filepath = path+allFiles[i][j]
    with open(filepath) as fp:  
      line = fp.readline()
      while line:
        sentences[j].append(line)
        line = fp.readline()
  #sentences contain 3 lists,every element of those lists contain a sentence from different evaluators
  for k in range(len(sentences[0])):
    #ignore comments
    if(sentences[0][k][0]!='#'):
      #for each sentence we extract the class from the 3 different evaluators
      term1=sentences[0][k][:4]
      term2=sentences[1][k][:4]
      term3=sentences[2][k][:4]
      #if 2 evaluators agree for the class of a sentence we accept it,otherwise we ignore the sentence
      if(term1==term2 or term1==term3):
        X.append(sentences[0][k][4:])
        Y.append(term1)
      elif(term2==term3):
        X.append(sentences[0][k][4:])
        Y.append(term2)
print(X[0])
X_preprossed=[]
for sen in X:
  sen=gensim.utils.simple_preprocess(remove_stopwords(sen))
  fullSen=''
  for i in range(len(sen)):
    fullSen=fullSen+' ' + sen[i]
  X_preprossed.append(fullSen)
  
print(X_preprossed[:5])
  
x_train, x_test, y_train, y_test = model_selection.train_test_split(X_preprossed, Y, random_state = 22,stratify=Y)

dictLabel={'AIMX':0,'OWNX':1,'CONT':4,'BASE':3,'MISC':2}

Ytrain=[dictLabel[label] for label in y_train]

# create the tokenizer
tokenizerTrain = Tokenizer()
# fit the tokenizer on the documents
tokenizerTrain.fit_on_texts(x_train)

max_length = max([len(s.split()) for s in x_train])

encoded_docsTrain = tokenizerTrain.texts_to_sequences(x_train)
# pad sequences
Xtrain = pad_sequences(encoded_docsTrain, maxlen=max_length, padding='post')
#print(Xtrain)
vocab_size = len(tokenizerTrain.word_index) + 1

Ytest=[dictLabel[label] for label in y_test]


# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(x_test)

encoded_docs = tokenizer.texts_to_sequences(x_test)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(Xtrain)

In [0]:


print(vocab_size,max_length)
print(Xtrain.shape)

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Dropout(0.5))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(16,activation='relu'))
#model.add(Flatten())
model.add(Dense(5, activation='softmax'))
print(model.summary())
# compile network
from keras import optimizers


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
model.predict(Xtest)

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Dropout(0.5))
model.add(Lambda(lambda x: mean(x, axis=1)))
#model.add(Flatten())
model.add(Dense(5, activation='sigmoid'))
print(model.summary())
# compile network
from keras import optimizers


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
for word in predicts:
  if word[1]>word[4]:
    print (word)

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Dropout(0.5))
model.add(Lambda(lambda x: mean(x, axis=1)))
#model.add(Flatten())
model.add(Dense(5, activation='linear'))
print(model.summary())
# compile network
from keras import optimizers


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [0]:

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(MaxPooling1D(pool_size=2,data_format='channels_last'))
model.add(Dropout(0.5))
model.add(Lambda(lambda x: mean(x, axis=1)))
#model.add(Flatten())
model.add(Dense(5, activation='sigmoid'))
print(model.summary())
# compile network
from keras import optimizers


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=5, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Dropout(0.5))
model.add(Lambda(lambda x: mean(x, axis=1)))
#model.add(Flatten())
model.add(Dense(5, activation='sigmoid'))
print(model.summary())
# compile network
from keras import optimizers


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=5, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [0]:
print(max_length)
# define model
model = Sequential()
model.add(firstModel.wv.get_keras_embedding(train_embeddings=False))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(5, activation='softmax'))
print("\nModel Summary:\n", model.summary())

           
# compile network
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [0]:
Xtrain